## Import

In [23]:
import pandas as pd
import numpy as np
import random
import os

from tqdm import tqdm
from statsmodels.tsa.arima.model import ARIMA

import warnings
warnings.filterwarnings("ignore")

In [24]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything(42) # Seed 고정

## Data Load

In [25]:
train = pd.read_csv('./train.csv')

# Arima

In [4]:
# 추론 결과를 저장하기 위한 dataframe 생성
results_df = pd.DataFrame(columns=['종목코드', 'final_return'])
# train 데이터에 존재하는 독립적인 종목코드 추출
unique_codes = train['종목코드'].unique()

# 각 종목코드에 대해서 모델 학습 및 추론 반복
for code in tqdm(unique_codes):
    
    # 학습 데이터 생성
    train_close = train[train['종목코드'] == code][['일자', '종가']]
    train_close['일자'] = pd.to_datetime(train_close['일자'], format='%Y%m%d')
    train_close.set_index('일자', inplace=True)
    tc = train_close['종가']
    
    # 모델 선언, 학습 및 추론
    model = ARIMA(tc, order=(2, 1, 2))
    model_fit = model.fit()
    predictions = model_fit.forecast(steps=15) # 향후 15개의 거래일에 대해서 예측
    
    # 최종 수익률 계산
    final_return = (predictions.iloc[-1] - predictions.iloc[0]) / predictions.iloc[0]
    
    # 결과 저장
    results_df = results_df.append({'종목코드': code, 'final_return': final_return}, ignore_index=True)
    
results_df['순위'] = results_df['final_return'].rank(method='first').astype('int') # 각 순위를 중복없이 생성
results_df

100%|██████████| 2000/2000 [08:30<00:00,  3.92it/s]


# Auto_Arima

In [27]:
import pandas as pd
import pmdarima as pm
from tqdm import tqdm

# 결과 저장을 위한 DataFrame 생성
results_df = pd.DataFrame(columns=['종목코드', 'final_return'])

# train 데이터에 존재하는 독립적인 종목코드 추출
unique_codes = train['종목코드'].unique()

# 각 종목코드에 대해서 모델 학습 및 추론 반복
for code in tqdm(unique_codes):
    
    # 학습 데이터 생성
    train_close = train[train['종목코드'] == code]['종가']
    train_close.index = pd.to_datetime(train[train['종목코드'] == code]['일자'], format='%Y%m%d')

    # 모델 선언 및 학습
    model = pm.auto_arima(train_close, error_action='ignore', trace=False,
                          suppress_warnings=True, maxiter=10,
                          seasonal=True, m=12)

    # 예측
    predictions = model.predict(n_periods=15)

    # 최종 수익률 계산
    final_return = (predictions.iloc[-1] - predictions.iloc[0]) / predictions.iloc[0]
    
    # 결과 저장
    results_df = results_df.append({'종목코드': code, 'final_return': final_return}, ignore_index=True)

results_df['순위'] = results_df['final_return'].rank(method='first').astype('int') # 각 순위를 중복없이 생성


100%|██████████| 2000/2000 [1:58:55<00:00,  3.57s/it]  


IntCastingNaNError: Cannot convert non-finite values (NA or inf) to integer

In [31]:
results_df.isnull().sum().sum()

9

# Prophet

In [16]:
#from fbprophet import Prophet
from prophet import Prophet

# 추론 결과를 저장하기 위한 dataframe 생성
results_df = pd.DataFrame(columns=['종목코드', 'final_return'])

# train 데이터에 존재하는 독립적인 종목코드 추출
unique_codes = train['종목코드'].unique()

# 각 종목코드에 대해서 모델 학습 및 추론 반복
for code in tqdm(unique_codes):
    
    # 학습 데이터 생성
    train_close = train[train['종목코드'] == code][['일자', '종가']]
    train_close = train_close.rename(columns={'일자': 'ds', '종가': 'y'})  # Prophet expects the date column to be 'ds' and the value column to be 'y'
    train_close['ds'] = pd.to_datetime(train_close['ds'], format='%Y%m%d')

    # 모델 선언, 학습 및 추론
    model = Prophet(daily_seasonality=True)  # Prophet model declaration
    model.fit(train_close)  # fit the model with your dataframe
    future = model.make_future_dataframe(periods=15)  # we need to specify the number of days we want to predict into the future
    predictions = model.predict(future)
    
    # 최종 수익률 계산
    final_return = (predictions['yhat'].iloc[-1] - predictions['yhat'].iloc[-15]) / predictions['yhat'].iloc[-15]
    
    # 결과 저장
    results_df = results_df.append({'종목코드': code, 'final_return': final_return}, ignore_index=True)

results_df['순위'] = results_df['final_return'].rank(method='first').astype('int') # 각 순위를 중복없이 생성


  0%|          | 0/2000 [00:00<?, ?it/s]15:21:49 - cmdstanpy - INFO - Chain [1] start processing
15:21:49 - cmdstanpy - INFO - Chain [1] done processing
  0%|          | 1/2000 [00:00<26:11,  1.27it/s]15:21:49 - cmdstanpy - INFO - Chain [1] start processing
15:21:49 - cmdstanpy - INFO - Chain [1] done processing
  0%|          | 2/2000 [00:01<20:05,  1.66it/s]15:21:50 - cmdstanpy - INFO - Chain [1] start processing
15:21:50 - cmdstanpy - INFO - Chain [1] done processing
  0%|          | 3/2000 [00:01<16:57,  1.96it/s]15:21:50 - cmdstanpy - INFO - Chain [1] start processing
15:21:50 - cmdstanpy - INFO - Chain [1] done processing
  0%|          | 4/2000 [00:02<15:21,  2.17it/s]15:21:50 - cmdstanpy - INFO - Chain [1] start processing
15:21:51 - cmdstanpy - INFO - Chain [1] done processing
  0%|          | 5/2000 [00:02<14:18,  2.32it/s]15:21:51 - cmdstanpy - INFO - Chain [1] start processing
15:21:51 - cmdstanpy - INFO - Chain [1] done processing
  0%|          | 6/2000 [00:02<13:14,  2.5

## Submit

In [18]:
sample_submission = pd.read_csv('./sample_submission.csv')
sample_submission

,종목코드,순위
0,A000020,1
1,A000040,2
2,A000050,3
3,A000070,4
4,A000080,5
...,...,...
1995,A375500,1996
1996,A378850,1997
1997,A383220,1998
1998,A383310,1999


In [19]:
baseline_submission = sample_submission[['종목코드']].merge(results_df[['종목코드', '순위']], on='종목코드', how='left')
baseline_submission

,종목코드,순위
0,A000020,686
1,A000040,1597
2,A000050,669
3,A000070,1334
4,A000080,309
...,...,...
1995,A375500,409
1996,A378850,1296
1997,A383220,985
1998,A383310,1446


In [20]:
baseline_submission.to_csv('baseline_prophet.csv', index=False)

In [ ]:
answer_df = pd.read_csv('Answer_real.csv', encoding = "CP949")
answer_df = answer_df[['종목코드','등락률']]
answer_df = answer_df.sort_values('종목코드')
answer_df


In [ ]:
prediction_df = pred_df[['종목코드','rate']]
prediction_df.rename(columns={'rate': '등락률'}, inplace=True)
prediction_df

In [ ]:
answer_df = answer_df[answer_df['종목코드'].isin(prediction_df['종목코드'])]
answer_df = answer_df.reset_index(drop = True)
answer_df

In [ ]:
from sklearn.metrics import accuracy_score, mean_absolute_error

# 정답 데이터프레임과 예측 결과 데이터프레임은 pandas DataFrame 형태라고 가정합니다.
# 등락률은 실수 형태의 값이라고 가정하고, 부호에 따라 'up' 또는 'down'으로 분류합니다.

# 등락률의 부호에 따른 분류 결과를 새로운 열로 추가합니다.
answer_df['trend'] = answer_df['등락률'].apply(lambda x: 'up' if x > 0 else 'down')
prediction_df['trend'] = prediction_df['등락률'].apply(lambda x: 'up' if x > 0 else 'down')

# 1. 전체 종목에 대한 Accuracy
total_accuracy = accuracy_score(answer_df['trend'], prediction_df['trend'])

# 변동률이 높은 상위 200개 및 하위 200개 종목 추출
top200_df = answer_df.sort_values(by='등락률', ascending=False).head(200)
bottom200_df = answer_df.sort_values(by='등락률', ascending=True).head(200)

# 2. 상위 200개 종목 및 하위 200개 종목에 대한 Accuracy
top200_accuracy = accuracy_score(top200_df['trend'], prediction_df.loc[top200_df.index, 'trend'])
bottom200_accuracy = accuracy_score(bottom200_df['trend'], prediction_df.loc[bottom200_df.index, 'trend'])

# 3. 상위 200개 종목 및 하위 200개 종목에 대한 MAE
# 이 경우 '등락률'은 실제 변동률을 예측한 연속적인 값이라고 가정합니다.
top200_mae = mean_absolute_error(top200_df['등락률'], prediction_df.loc[top200_df.index, '등락률'])
bottom200_mae = mean_absolute_error(bottom200_df['등락률'], prediction_df.loc[bottom200_df.index, '등락률'])

print("Total Accuracy: ", total_accuracy)
print("Top 200 Accuracy: ", top200_accuracy)
print("Bottom 200 Accuracy: ", bottom200_accuracy)
print("Top 200 MAE: ", top200_mae)
print("Bottom 200 MAE: ", bottom200_mae)
